# Setup

In [ ]:
import pandas as pd
from tqdm import tqdm
import re
import json
import nltk
import numpy as np
import matplotlib.pyplot as plt

from random import shuffle

from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import DataCollatorForLanguageModeling, AutoTokenizer, TFAutoModelForCausalLM, AdamWeightDecay
from os.path import join, exists

In [ ]:
from lib.BBData import character_dict, model_name
from lib.BBDataLoad import dialogpt_preprocess_function, load_char_df, get_chatbot_predictions, merge_df_for_metrics

from lib.wip.frequency import sentence_preprocess, freq_pairwise_sim, filter_by_weights, get_word_frequency, get_tfidfs, FrequencyChatbotClassifier
from sklearn.metrics import confusion_matrix

In [ ]:
characters = list(character_dict.keys())
characters.remove('Default')

mass_value = 0.3

In [ ]:
# Mount google drive, if in Colaboratory environment
import os
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    base_folder = '/content/drive/My Drive/unibo/NLP_project/BarneyBot'
    os.system("pip install datasets")
    os.system("pip install transformers")
    os.system("pip install rouge_score")
    os.system("pip install -U sentence-transformers")
else:
    # base_folder = os.getcwd()
    base_folder = '..'

out_folder = os.path.join(base_folder, 'Data', 'Characters')

# Data Loading

In [ ]:
character_docs = dict()
for character in characters:
    df = pd.read_csv(os.path.join(out_folder, character, f'{character}.csv'))
    character_docs[character] = df['response'].tolist()

In [ ]:
for character in characters:
    for i in tqdm(range(len(character_docs[character]))):
        sentence, relevant = sentence_preprocess(character_docs[character][i])
        if relevant:
            character_docs[character][i] = sentence

In [ ]:
test_size = 0.33
character_docs_train = {}
character_docs_test = {}
for c in characters:
    shuffle(character_docs[c])
    end_idx = int(len(character_docs[c]) * test_size)
    character_docs_train[c] = character_docs[c][end_idx:]
    character_docs_test[c] = character_docs[c][:end_idx]

# Word Frequency

Loading word frequencies

In [ ]:
wordfreqs_train = dict()
for character in tqdm(characters):
    wordfreqs_train[character] = get_word_frequency(' '.join(character_docs_train[character]), f_sorted=True)

wordfreqs_test = dict()
for character in tqdm(characters):
    wordfreqs_test[character] = get_word_frequency(' '.join(character_docs_test[character]), f_sorted=True)

In [ ]:
wordfreqs_reduced_train = dict()
for character in characters:
    wordfreqs_reduced_train[character] = filter_by_weights(wordfreqs_train[character], mass=0.3)

wordfreqs_reduced_test = dict()
for character in characters:
    wordfreqs_reduced_test[character] = filter_by_weights(wordfreqs_test[character], mass=0.3)

Loading TF-IDF matrix

In [ ]:
tfidf_vectorizer = TfidfVectorizer(input='content', stop_words='english')

In [ ]:
tfidfs_train = get_tfidfs([' '.join(character_docs_train[character]) for character in characters], characters, tfidf_vectorizer)

In [ ]:
tfidfs_reduced_train = dict()
for character in characters:
    tfidfs_reduced_train[character] = filter_by_weights(tfidfs_train[character], mass=mass_value)

# WordCloud Plot (Example)

In [ ]:
from lib.BBVisualizations import BBVisualization
BBVisualization.load_visualization("wordcloud").plot(freqdict=tfidfs_reduced_train['Barney'])

# Frequency Pairwise Similarity (Example)

In [ ]:
freq_pairwise_sim(wordfreqs_reduced_train['Fry'], wordfreqs_reduced_test['Fry'])

# Test Classifier Performances Against Test Set

In [ ]:
wf_classifier = FrequencyChatbotClassifier(characters, mode='word frequency')
wf_classifier.train(list(character_docs_train.values()))

y_true = range(0, len(characters))
y_pred = [np.argmax(list(wf_classifier.predict(character_docs_test[character], mass=0.3).values()))
          for character in characters]

print('Word Frequency classifier test accuracy: {:.2f}'.format(
             sum([y_pred[i] == y_true[i] for i in range(len(y_true))]) / len(y_true)))
confusion_matrix(y_true, y_pred)

In [ ]:
tfidf_classifier = FrequencyChatbotClassifier(characters, mode='tf-idf')
tfidf_classifier.train(list(character_docs_train.values()))

y_true = range(0, len(characters))
y_pred = [np.argmax(list(tfidf_classifier.predict(character_docs_test[character], mass=0.3).values()))
          for character in characters]

print('TF-IDF classifier test accuracy: {:.2f}'.format(
             sum([y_pred[i] == y_true[i] for i in range(len(y_true))]) / len(y_true)))
confusion_matrix(y_true, y_pred)

# Test Classifiers On Chatbots (Not Cached!)

In [ ]:
tfidf_classifier = FrequencyChatbotClassifier(characters, mode='tf-idf')
tfidf_classifier.train(list(character_docs.values()))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=join("..", "cache"))
tokenizer.pad_token = '#'
data_collator = DataCollatorForLanguageModeling(mlm=False, tokenizer=tokenizer, return_tensors='tf')

In [ ]:
### create dataset
n_tests = 10
doc_test = {c:[] for c in characters}
batch_size = 32
override_predictions = True
predictions = {c:[] for c in characters}
raw_predictions = {c:[] for c in characters}
print('Creating dataset...')
if n_tests > 1 and not override_predictions:
    raise Exception('must override previous predictions if you need more tests')

for character in characters:
    print('Character: ', character)
    for i in range(n_tests):
        print(f'Test {i+1}/{n_tests}')
        character_checkpoint = join(out_folder, character, character_dict[character]['checkpoint_folder'])
        model_chatbot = TFAutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=character_checkpoint) if override_predictions else None
        if model_chatbot:
            model_chatbot.compile(optimizer=AdamWeightDecay(learning_rate=2e-5))

        character_hg = load_char_df(character, base_folder)
        # This transform in a sequence of tokens ours dataset
        tokenized_character_hg = character_hg.map(lambda row: dialogpt_preprocess_function(row, tokenizer), batched=False)

        # Define tensorflow datasets
        encoded_test_set = tokenized_character_hg["test"].to_tf_dataset(
            columns=["input_ids", "attention_mask", "labels"],
            shuffle=False,
            batch_size=batch_size,
            collate_fn=data_collator,
        )

        # Takes the testset as sample question 
        sample_questions = character_hg['test']['context/0']

        # Sampling generation method
        predictions_sampling = get_chatbot_predictions(
            sample_questions,
            model_chatbot,
            character_dict[character]['prediction_filename'] + '_sampling.json',
            "Sampling",
            character,
            tokenizer,
            base_folder,
            override_predictions=override_predictions
        )
                                                    
        sentences = merge_df_for_metrics(character_hg['test'], None, None, predictions_sampling, tokenizer)['prd_sampling'].tolist()
        doc_test[character].append([sentence_preprocess(s) for s in sentences])

In [ ]:
print('Classification...')
for c in range(len(characters)):
    print('Character: ', characters[c])
    for doc in doc_test[characters[c]]:
        prediction = tfidf_classifier.predict(doc, mass=mass_value)
        raw_predictions[characters[c]].append(prediction)
        predictions[characters[c]].append(
            int(max(prediction, key=prediction.get) == characters[c])
        )

In [ ]:
# raw_predictions

In [ ]:
# predictions

In [ ]:
print('TF-IDF classifier test accuracy: {:.2f}'.format(sum([char_pred[-1] for char_pred in predictions.values()])/len(predictions)))

In [ ]:
### save predictions
append_predictions = True
override_predictions = False
predictions_file = join('..', 'Data', 'tfidf_predictions.json')

if append_predictions and exists(predictions_file):
    with open(predictions_file, 'r', encoding='utf-8') as file:
        predictions_dict = json.load(file)
elif override_predictions or not exists(predictions_file):
    predictions_dict = {'one_hot':{c:[] for c in characters}, 'raw_predictions': {c:[] for c in characters}}
else:
    raise NotImplementedError
    
for c in characters:
    predictions_dict['one_hot'][c] += predictions[c]
    predictions_dict['raw_predictions'][c] += raw_predictions[c]

with open(predictions_file, 'w', encoding='utf-8') as file:
    json.dump(predictions_dict, file)

In [ ]:
predictions_dict